In [53]:
import pandas as pd

In [54]:
folder = '.\\input_files\\other_schema\\'

file = '202552602_Mercy Medical Center Dyersville_standardcharges.xlsx'

In [55]:
df = pd.read_excel(folder + file, skiprows=1, dtype=str)

In [56]:
df.rename(columns={
    'Code': 'code',
    'Description': 'description',
    'Type': 'setting',
}, inplace=True)

In [57]:
cols = df.columns.tolist()
id_vars = cols[:3]
value_vars = cols[3:]

df = pd.melt(df, id_vars=id_vars, value_vars=value_vars, var_name='payer_name', value_name='standard_charge')

In [58]:
df['code'] = df['code'].str.strip()

In [59]:
df.loc[(~df['code'].isna()) & (df['code'].str.match(r'^[A-Z][0-9]{4}$|^[0-9]{5}$|^[0-9]{4}[A-Z]$')), 'hcpcs_cpt'] = df['code']
df.loc[(~df['code'].isna()) & (df['code'].str.len() == 3), 'ms_drg'] = df['code']

In [63]:
mapping = {
    'Gross charge': 'gross',
    'Discounted cash price': 'cash',
    'De-identified min contracted rate': 'min',
    'De-identified max contracted rate': 'max'
}

df['payer_category'] = df['payer_name'].map(mapping).fillna('payer')

In [61]:
df.loc[(~df['payer_name'].str.contains('De-identified') & (df['payer_name'].str.contains('-'))), 'plan_name'] = df['payer_name'].str.split('-').str[-1]

In [66]:
df['setting'] = df['setting'].str.lower()

In [69]:
mapping = {'421470935_mercyone-newton-medical-center_standardcharges.csv': '160032',
    '420758901_MercyOne Cedar Falls Medical Center_standardcharges.xlsx': '160040',
    '311373080_Mercy Medical Center North Iowa_standardcharges.xlsx': '160064',
    '421264647_MercyOne Waterloo Medical Center_standardcharges.xlsx': '160067',
    '421437483_Mercy Medical Center Dubuque_standardcharges.xlsx': '160069',
    '421336618_Mercy Medical Center Clinton_standardcharges.xlsx': '160080',
    '420680448_mercyone-des-moines-medical-center_standardcharges.xlsx': '160083',
    '311407377_Mercy Medical Center Siouxland_standardcharges.xlsx': '160153',
    '421500277_MercyOne Primghar Medical Center_standardcharges.xlsx': '161300',
    '420818642_MercyOne Elkader Medical Center_standardcharges.xlsx': '161319',
    '421178403_MercyOne Oelwein Medical Center_standardcharges.xlsx': '161338',
    '420680308_mercyone-centerville-medical-center_standardcharges.csv': '161377',
    '202552602_Mercy Medical Center Dyersville_standardcharges.xlsx': '161378'}

hospital_id = mapping[file]
df['hospital_id'] = hospital_id

In [70]:
df

,code,description,setting,payer_name,standard_charge,hcpcs_cpt,ms_drg,payer_category,plan_name,hospital_id
0,10060,SIMPLE OR SINGLE DRAINAGE OF SKIN ABSCESS,outpatient,Gross charge,705,10060,NaN,gross,NaN,161378
1,11730,SIMPLE SEPARATION OF FINGERNAIL OR TOENAIL FRO...,outpatient,Gross charge,1455,11730,NaN,gross,NaN,161378
2,12001,"SIMPLE REPAIR OF SURFACE WOUND OF SCALP, NECK,...",outpatient,Gross charge,395.43,12001,NaN,gross,NaN,161378
3,12002,"SIMPLE REPAIR OF SURFACE WOUND OF SCALP, NECK,...",outpatient,Gross charge,396.8,12002,NaN,gross,NaN,161378
4,12004,"SIMPLE REPAIR OF SURFACE WOUND OF SCALP, NECK,...",outpatient,Gross charge,404,12004,NaN,gross,NaN,161378
...,...,...,...,...,...,...,...,...,...,...
17859,64483,INJECTION OF ANESTHETIC AND/OR STEROID DRUG IN...,outpatient,Blue Cross-Medicare Advantage HMO,NaN,64483,NaN,payer,Medicare Advantage HMO,161378
17860,66821,REMOVAL OF RECURRING CATARACT IN LENS CAPSULE ...,outpatient,Blue Cross-Medicare Advantage HMO,NaN,66821,NaN,payer,Medicare Advantage HMO,161378
17861,93000,ROUTINE ELECTROCARDIOGRAM (ECG) USING AT LEAST...,outpatient,Blue Cross-Medicare Advantage HMO,NaN,93000,NaN,payer,Medicare Advantage HMO,161378
17862,93452,INSERTION OF TUBE IN LEFT HEART CHAMBERS FOR D...,outpatient,Blue Cross-Medicare Advantage HMO,NaN,93452,NaN,payer,Medicare Advantage HMO,161378
